# Kalman filter vs complementary filter for IMU-based pose estimation


An Inertial measurement unit (IMU) comprises accelerometers, gyroscopes and 
magnetometers and allows to estimate the pose of a rigid body. The gyroscope 
provides measurements of the angular velocity and from the measurements of the 
accelerometer and of the magnetometer one can extract measurements of the three 
angles (Euler angles according to a given convention) characterizing the attitude. 
In a simple linear model the kynematic equations for the three angles are decoupled. 
In particular, one can simply consider the following model

$$\dot{\lambda}=\omega$$

$$\dot{\omega}=\frac{m}{I_{\omega}}$$

where $\lambda \ \in \ R$, denotes any of the three angles describing the 
attitude (pitch, roll, or yaw), and $\omega \ \in \ R, \ m, \ I_{\omega}$ denote 
the angular velocity, the torque and the moment of inertial about the corresponding 
axis. Using a simple Euler discretization, we obtain

$$\lambda_{k+1}=\lambda_{k}+\tau \omega_{k}$$

$$\omega_{k+1}=\omega_{k}+\tau \frac{m_{k}}{I_{\omega}}$$

where for a given function of time $x(t)$ we use the notation $x_k := x(k\tau)$ 
and $\tau$ is the sampling time. Additionally, the following measurements are 
available

$$y_{\lambda, k}=\lambda_{k}+n_{k}$$

$$y_{\omega, k}=\omega_{k}+v_{k}$$

where $\left\{n_{k} | k \in \mathbb{N}_{0}\right\}$ and $\left\{v_{k} | k 
\in \mathbb{N}_{0}\right\}$ are independent and identically distributed sequences 
of zero mean Gaussian white noise with variance at time $k$, $N$ and $V$ , respectively. 
Assume that the torque $m_k$ is the sum of a known component $u_k$ and a disturbance 
signal $\omega _k$

$$m_k=u_k+\omega _k$$

where $\{\omega _k | k\ \in \mathbb{N}_{0} \}$ is an independent and identically 
distributed sequence of zero mean Gaussian white noise with covariance $W$. 

The goal is to find an estimator providing estimates $\hat{\lambda}_{k}$ of 
the angle $\lambda_{k}$, for $k \in \mathbb{N}_{0}$, such that the following 
asymptotic variance is minimized

\begin{equation*}
\lim _{k \rightarrow \infty} \mathbb{E}\left[\left(\lambda_{k}-\hat{\lambda}_{k}\right)^{2}\right]
\label{eq:asym_var} \tag{1}
\end{equation*}

Two of the most known methods are the standard Kalman filter and the complementary 
filter. The complementary filter can be written, in continuous-time, as

$$\dot{\hat{\lambda}}=y_{\omega}+\alpha\left(y_{\lambda}-\hat{\lambda}\right)$$

where $\alpha$ is a given constant; in discrete-time it takes the form

$$\hat{\lambda}_{k+1}=\hat{\lambda}_{k}+\tau y_{\omega, k}+\tau \alpha\left(y_{\lambda, 
k}-\hat{\lambda}_{k}\right)$$

One of the strong features of the complimentary filter is that it does not 
assume knowledge of the torque $m_k$, i.e., the function $u_k$ does not need 
to be known.

The goal of this live script is to compare the performance of the optimal 
estimator that minimizes $\eqref{eq:asym_var}$ (Kalman filter) and the optimal complimentary filter 
corresponding to the optimal value of $\alpha$ that minimizes (2). Since for 
the complimentary filter, $\hat{\lambda_{k}}$ is a function of $\left\{y_{0}, 
\ldots, y_{k-1}\right\}$, where $y_{k}=\left[y_{\lambda, k} \quad y_{\omega, 
k}\right]^{\top}$, for the Kalman filter consider the estimate 

$$\hat{\lambda}_{k}=\hat{\lambda}_{k | k-1}=\mathbb{E}\left[\lambda_{k}|\left\{y_{0}, \ldots, y_{k-1}, u_{0}, \ldots, u_{k-1}\right\}\right]$$

The Matlab function `attitudeestimetors` below computes the asymptotic variances 
of the Kalman and the complementary filter, as well as the optimal value of 
$\alpha$ of the complementary filter which minimizes (2). These values are noted 
in the function as varkf, varcf and optalpha respectively. The inputs to the 
function are the inertia Iomega ($I_{\omega}$), the sampling time $\tau$, the 
variances $N$ and $V$ and the covariance $W$. 

These values can be obtained theoretically as follows. For the Kalman filter 
approach, we can write the problem in the standard state-space canonical form

$$\underbrace{\left[ \matrix{\lambda_{k+1} \cr \omega_{k+1}} \right]}_{x_{k+1}}=\underbrace{\left[ 
\matrix{1 & \tau \cr 0 & 1} \right]}_A\underbrace{\left[ \matrix{\lambda_{k} 
\cr \omega_{k}} \right]}_{x_k}+\underbrace{\left[ \matrix{0 \cr \tau \frac{1}{I_{\omega}}} 
\right]}_{B}u_{k}+\underbrace{\left[ \matrix{0 \cr \tau \frac{m_{k}}{I_{\omega}}} 
\right]}_{w_k}$$

$$\underbrace{\left[ \matrix{y_{\lambda, k} \cr y_{\omega, k}} \right]}_{y_{k}} 
= \underbrace{I}_C\underbrace{\left[ \matrix{ \lambda_k \cr \omega_ k} \right]}_{x_{k}}+\underbrace{\left[ 
\matrix{n_{k} \cr v_{k}} \right]}_{\bar{v}_{k}} $$

Note that the covariance matrix of the disturbance model is

$$E[w_kw_k^T] = \left[ \matrix{ 0 & 0 \cr 0 & \frac{\tau^2}{I_\omega}W}  \right]$$

and the covariance of the output noise is

$E[\bar{v_k}\bar{v_k}^T]=\left[\matrix{N & 0\cr 0 & V}\right]$.

The Kalman filter framework provides the error covariance matrix

$$E[(x_k-\hat{x}_k)(x_k-\hat{x}_k)^T]=E[(x_k-\hat{x}_k)(x_k-\hat{x}_k)^T] 
= E[\left[ \matrix{\lambda_k-\hat{\lambda}_k \cr \omega_k-\hat{\omega}_k } \right] 
\left[ \matrix{\lambda_k-\hat{\lambda}_k \cr \omega_k-\hat{\omega}_k } \right]^T 
]$$

and the stationary Kalman filter provides the asymptotic value as $k\rightarrow 
\infty$. The component in the first row first column is the desired value in 
$\eqref{eq:asym_var}$.

In order to obtain the optimal value of $\alpha$ for the complementary filter, 
we notice that we can view the complementary filter as a Kalman filter but for 
a simplified state model considering only $\lambda_k$

$$\lambda_{k+1}=\lambda_{k}+\tau \omega_{k}\rightarrow \lambda_{k+1}=\lambda_{k}+\tau 
y_{\omega,k}-\tau v_k$$

where $y_{\omega,k}$ is now seen as a known input. The output is now only

$$y_{\lambda, k}=\lambda_{k}+n_{k}.$$

The covariance of the state disturbances is $E[(\tau v_k)^2]=\tau^2V$ and 
of the output noise is $E[n_k^2]=N$. The asymptotic Kalman filter for this simplified 
model is

$$\hat{\lambda}_{k+1}=\hat{\lambda}_{k}+\tau y_{\omega, k}+L\left(y_{\lambda, 
k}-\hat{\lambda}_{k}\right)$$

for an optimal gain $L$ which minimizes the state error (co)variance $\eqref{eq:asym_var}$. 
Noticing that the expression for the complementary filter is exactly the same 
but with $L$ replaced by $\tau \alpha$, it suffices to divide $L$ by $\tau$ 
to obtain the optimal $\alpha$.

An example is given next. Note that the error output variance is smaller for 
the Kalman filter since it is the optimal filter.


In [ ]:
import numpy as np
from course_functions import kalman

In [ ]:
def attitudeestimators(Iomega,tau,N,V,W):

    # Kalman filter
    A = np.array([[1, tau],[0, 1]])
    B = np.array([[0],[tau/Iomega]])
    C = np.array([[1, 0],[0, 1]])
    n = 2
    
    G = np.array(([[1, 0],[  0, 1]]))
    QN = np.diag(np.array([0, (tau/Iomega)**2*W]))
    RN = np.diag(np.array([N, V]))
    L, P1_, ED = kalman(A, G, C, QN, RN, continuous=False)[:3]
    varkf = np.array([[1,0]]) @ P1_ @ np.array([[1],[0]])

    # complementary filter
    A_ = 1 
    B_ = tau
    C_ = 1
    G_ = np.array([1])
    QN_ = tau*tau*V
    RN_ = N
    L, varcf, E = kalman(A_, G_, C_, QN_, RN_, continuous=False)[:3]

    optalpha = L/tau
    return varcf, varkf, optalpha

In [ ]:
#Define the input values and call the functetion
tau    = 0.1
N      = 1
V      = 1
W      = 1
Iomega = 0.1
varcf,varkf,optalpha  = attitudeestimators(Iomega,tau,N,V,W)
varcf

In [ ]:
varkf

In [ ]:
optalpha